In [2]:
import pandas as pd
import json
import requests
import folium

In [7]:
places = pd.read_csv('업체정보.csv')

In [8]:
places

,업체명,업종,주소,전화번호,썸네일
0,강군초밥,"초밥,롤",충북 충주시 연원로 28 1층 강군초밥,043-845-3412,https://search.pstatic.net/common/?autoRotate=...
1,장수골,한식,충북 충주시 대소원면 쇠실로 984,043-846-4189,https://search.pstatic.net/common/?autoRotate=...
2,원조영덕가든,한식,충북 충주시 노은면 감노로 1595-1 원조영덕가든,043-853-2264,https://search.pstatic.net/common/?autoRotate=...
3,충주댐198510,"카페,디저트",충북 충주시 충주호수로 586,0507-1456-1985,https://search.pstatic.net/common/?autoRotate=...
4,비건채식뷔페 빛가람,"채식,샐러드뷔페",충북 충주시 신대2길 26 비건채식뷔페 빛가람,043-842-3333,https://search.pstatic.net/common/?autoRotate=...
...,...,...,...,...,...
1583,엘몽끄,"카페,디저트","경기 안양시 만안구 병목안로 20 스페이스타워 1층,2층",031-441-7337,https://search.pstatic.net/common/?autoRotate=...
1584,리얼케익 안양범계점,케이크전문,경기 안양시 동안구 평촌대로227번길 30 1층 리얼케익,0507-1311-8393,https://search.pstatic.net/common/?autoRotate=...
1585,다정김밥,김밥,경기 안양시 만안구 안양로 316 지하2층 비220호,0507-1364-9868,https://search.pstatic.net/common/?autoRotate=...
1586,페어리우즈 카페,브런치카페,경기 안양시 동안구 시민대로131번길 10 1층 페어리우즈,0507-1320-4546,https://search.pstatic.net/common/?autoRotate=...


In [9]:
api_key = '98b6364b112863dd8d4a6d308bac6783'

In [10]:
def addr_to_lat_lon(addr):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
    headers = {"Authorization": "KakaoAK " + api_key}
    response = requests.get(url, headers=headers)
    result = json.loads(response.text)
    
    if result['documents']:
        match_first = result['documents'][0]['address']
        if match_first is not None:
            return float(match_first['y']), float(match_first['x'])  # latitude, longitude
    return None, None  # 검색 결과가 없거나 주소가 None인 경우

In [11]:
# 데이터프레임의 address 열에 함수 적용
places[['latitude', 'longitude']] = places['주소'].apply(lambda x: pd.Series(addr_to_lat_lon(x)))

In [12]:
places

,업체명,업종,주소,전화번호,썸네일,latitude,longitude
0,강군초밥,"초밥,롤",충북 충주시 연원로 28 1층 강군초밥,043-845-3412,https://search.pstatic.net/common/?autoRotate=...,36.988591,127.930922
1,장수골,한식,충북 충주시 대소원면 쇠실로 984,043-846-4189,https://search.pstatic.net/common/?autoRotate=...,36.972495,127.820255
2,원조영덕가든,한식,충북 충주시 노은면 감노로 1595-1 원조영덕가든,043-853-2264,https://search.pstatic.net/common/?autoRotate=...,37.042982,127.774002
3,충주댐198510,"카페,디저트",충북 충주시 충주호수로 586,0507-1456-1985,https://search.pstatic.net/common/?autoRotate=...,37.011183,127.978671
4,비건채식뷔페 빛가람,"채식,샐러드뷔페",충북 충주시 신대2길 26 비건채식뷔페 빛가람,043-842-3333,https://search.pstatic.net/common/?autoRotate=...,36.947425,127.898665
...,...,...,...,...,...,...,...
1583,엘몽끄,"카페,디저트","경기 안양시 만안구 병목안로 20 스페이스타워 1층,2층",031-441-7337,https://search.pstatic.net/common/?autoRotate=...,37.398955,126.918372
1584,리얼케익 안양범계점,케이크전문,경기 안양시 동안구 평촌대로227번길 30 1층 리얼케익,0507-1311-8393,https://search.pstatic.net/common/?autoRotate=...,37.391438,126.953821
1585,다정김밥,김밥,경기 안양시 만안구 안양로 316 지하2층 비220호,0507-1364-9868,https://search.pstatic.net/common/?autoRotate=...,37.400531,126.920602
1586,페어리우즈 카페,브런치카페,경기 안양시 동안구 시민대로131번길 10 1층 페어리우즈,0507-1320-4546,https://search.pstatic.net/common/?autoRotate=...,37.388639,126.946557


In [13]:
null_rows = places[places[['latitude', 'longitude']].isnull().any(axis=1)]

In [14]:
null_rows

,업체명,업종,주소,전화번호,썸네일,latitude,longitude
30,먹보한우 파주점,소고기구이,경기 파주시 지목로 43 101~105호,0507-1362-9955,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
33,내사랑동까스,돈가스,경기 파주시 송학말길 65 1충,031-945-4800,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
45,금성회관 파주야당점,"육류,고기요리","경기 파주시 소리천로 39 117, 118, 119호",0507-1399-2272,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
60,보약정 추어명가 추어탕 돈까스,한식,"경기 평택시 고덕여염9길 20 101, 102, 103, 104, 107, 108,...",0507-1358-2236,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
120,수복삼계탕 일산점,"백숙,삼계탕","경기 고양시 일산동구 장백로 200 114, 115, 116호",031-932-9327,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
221,르빵드비,"카페,디저트","경기 하남시 서하남로 436 1,2,3층",0507-1430-1129,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
234,소주물 미사본점,"육류,고기요리","경기 하남시 미사강변중앙로 181 106, 107호",031-796-0980,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
254,품육참숯소갈비살전문점 화성시청점,"육류,고기요리",경기 화성시 남양읍 남양로 617 1충,0507-1422-6890,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
272,옥이떡볶이,떡볶이,경기 화성시 목동 393-10,0507-1415-4110,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN
399,곤드레찜드레 용인점,"육류,고기요리","경기 용인시 처인구 명지로 31 104, 105호",0507-1394-6777,https://search.pstatic.net/common/?autoRotate=...,NaN,NaN


In [36]:
null_rows.isnull().sum()

업체명          0
업종           2
주소           2
전화번호         2
썸네일          2
latitude     0
longitude    0
dtype: int64

In [25]:
miss_latitude = [37.7264152,37.7067086,37.7163925,37.0391219,37.6525709,37.5216397,37.5625350,37.1949681,37.183310,37.2332457,37.3516282,37.3925207,
                35.8142576,33.4643204,37.5836035,36.1299759,36.1960000,36.8907492,36.8841349,37.9030036,37.4989175, 37.2461035,37.2654128,37.2807338,
                37.3410195,37.3840638,37.3362014,37.2822636,37.3104079,37.3181300,37.2804909,37.3927936]

In [24]:
miss_longitude = [126.6960509,126.7616044,126.7619732,127.0488304,126.7764242,127.1882042,127.1902085,126.8220652,127.1207578,127.1889302,127.0694440,
                  126.9880719,127.1513311,126.3090018,127.1233685,127.1201792,127.2947570,126.8233683,126.6180931,127.0594271,126.7762668,126.9775483,
                  127.0322937,127.0589033,126.6880130,126.8357619,126.8102084,126.8281717,126.8284574,126.8365824,126.5683656,126.9530641]

In [26]:
null_rows.loc[:, 'latitude'] = miss_latitude
null_rows.loc[:, 'longitude'] = miss_longitude

In [27]:
null_rows

,업체명,업종,주소,전화번호,썸네일,latitude,longitude
30,먹보한우 파주점,소고기구이,경기 파주시 지목로 43 101~105호,0507-1362-9955,https://search.pstatic.net/common/?autoRotate=...,37.726415,126.696051
33,내사랑동까스,돈가스,경기 파주시 송학말길 65 1충,031-945-4800,https://search.pstatic.net/common/?autoRotate=...,37.706709,126.761604
45,금성회관 파주야당점,"육류,고기요리","경기 파주시 소리천로 39 117, 118, 119호",0507-1399-2272,https://search.pstatic.net/common/?autoRotate=...,37.716392,126.761973
60,보약정 추어명가 추어탕 돈까스,한식,"경기 평택시 고덕여염9길 20 101, 102, 103, 104, 107, 108,...",0507-1358-2236,https://search.pstatic.net/common/?autoRotate=...,37.039122,127.048830
120,수복삼계탕 일산점,"백숙,삼계탕","경기 고양시 일산동구 장백로 200 114, 115, 116호",031-932-9327,https://search.pstatic.net/common/?autoRotate=...,37.652571,126.776424
221,르빵드비,"카페,디저트","경기 하남시 서하남로 436 1,2,3층",0507-1430-1129,https://search.pstatic.net/common/?autoRotate=...,37.521640,127.188204
234,소주물 미사본점,"육류,고기요리","경기 하남시 미사강변중앙로 181 106, 107호",031-796-0980,https://search.pstatic.net/common/?autoRotate=...,37.562535,127.190208
254,품육참숯소갈비살전문점 화성시청점,"육류,고기요리",경기 화성시 남양읍 남양로 617 1충,0507-1422-6890,https://search.pstatic.net/common/?autoRotate=...,37.194968,126.822065
272,옥이떡볶이,떡볶이,경기 화성시 목동 393-10,0507-1415-4110,https://search.pstatic.net/common/?autoRotate=...,37.183310,127.120758
399,곤드레찜드레 용인점,"육류,고기요리","경기 용인시 처인구 명지로 31 104, 105호",0507-1394-6777,https://search.pstatic.net/common/?autoRotate=...,37.233246,127.188930


In [28]:
for i in range(len(places)):
    if places['주소'][i] in null_rows['주소'].values:
        places.loc[i, ['latitude', 'longitude']] = null_rows[null_rows['주소'] == places['주소'][i]][['latitude', 'longitude']].values[0]

In [39]:
places.isnull().sum()

업체명          0
업종           2
주소           2
전화번호         9
썸네일          2
latitude     2
longitude    2
dtype: int64

In [42]:
df_cleaned = places.dropna(subset=['업체명'])

In [43]:
df_cleaned.isnull().sum()

업체명          0
업종           2
주소           2
전화번호         9
썸네일          2
latitude     2
longitude    2
dtype: int64

In [46]:
places.to_excel('placesfinal.xlsx', index=False)

In [33]:
test = pd.read_csv("placesfinal.csv")

In [34]:
test

,업체명,업종,주소,전화번호,썸네일,latitude,longitude
0,강군초밥,"초밥,롤",충북 충주시 연원로 28 1층 강군초밥,043-845-3412,https://search.pstatic.net/common/?autoRotate=...,36.988591,127.930922
1,장수골,한식,충북 충주시 대소원면 쇠실로 984,043-846-4189,https://search.pstatic.net/common/?autoRotate=...,36.972495,127.820255
2,원조영덕가든,한식,충북 충주시 노은면 감노로 1595-1 원조영덕가든,043-853-2264,https://search.pstatic.net/common/?autoRotate=...,37.042982,127.774002
3,충주댐198510,"카페,디저트",충북 충주시 충주호수로 586,0507-1456-1985,https://search.pstatic.net/common/?autoRotate=...,37.011183,127.978671
4,비건채식뷔페 빛가람,"채식,샐러드뷔페",충북 충주시 신대2길 26 비건채식뷔페 빛가람,043-842-3333,https://search.pstatic.net/common/?autoRotate=...,36.947425,127.898665
...,...,...,...,...,...,...,...
1583,엘몽끄,"카페,디저트","경기 안양시 만안구 병목안로 20 스페이스타워 1층,2층",031-441-7337,https://search.pstatic.net/common/?autoRotate=...,37.398955,126.918372
1584,리얼케익 안양범계점,케이크전문,경기 안양시 동안구 평촌대로227번길 30 1층 리얼케익,0507-1311-8393,https://search.pstatic.net/common/?autoRotate=...,37.391438,126.953821
1585,다정김밥,김밥,경기 안양시 만안구 안양로 316 지하2층 비220호,0507-1364-9868,https://search.pstatic.net/common/?autoRotate=...,37.400531,126.920602
1586,페어리우즈 카페,브런치카페,경기 안양시 동안구 시민대로131번길 10 1층 페어리우즈,0507-1320-4546,https://search.pstatic.net/common/?autoRotate=...,37.388639,126.946557
